# Ideation

More backtesting, except NHL instead and NFL and moneyline instead of the spread/puckline.

In [48]:
import pandas as pd
from collections import deque
import datetime as dt

In [2]:
nhl_odds = pd.read_csv('combined_nhl_odds.csv')
nhl_odds

,Unnamed: 0,Date,AwayTeam,Away1st,Away2nd,Away3rd,AwayFinal,MlAwayOpen,MlAwayClose,OpenOU,...,Year,HomeTeam,Home1st,Home2nd,Home3rd,HomeFinal,MlHomeOpen,MlHomeClose,OpenUOdds,CloseUOdds
0,0,929,Anaheim,0,0,1,1,-155.0,-123.0,6.0,...,2008,LosAngeles,1.0,1.0,2.0,4.0,135.0,103.0,-120.0,-115.0
1,2,930,LosAngeles,0,1,0,1,115.0,132.0,6.0,...,2008,Anaheim,2.0,2.0,0.0,4.0,-135.0,-152.0,-105.0,-132.0
2,4,1003,Anaheim,0,2,0,2,165.0,190.0,5.5,...,2008,Detroit,1.0,0.0,1.0,3.0,-185.0,-230.0,-125.0,-140.0
3,6,1003,Montreal,1,0,1,3,130.0,140.0,6.0,...,2008,Carolina,1.0,0.0,1.0,2.0,-150.0,-160.0,-115.0,-130.0
4,8,1003,Ottawa,2,0,1,4,-120.0,-125.0,6.0,...,2008,Toronto,2.0,1.0,0.0,3.0,100.0,105.0,100.0,115.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17528,1894,628,Montreal,0,1,0,1,185.0,180.0,5.0,...,2021,TampaBay,1.0,1.0,3.0,5.0,-225.0,-200.0,100.0,105.0
17529,1896,630,Montreal,0,1,0,1,180.0,177.0,5.0,...,2021,TampaBay,0.0,2.0,1.0,3.0,-220.0,-197.0,100.0,100.0
17530,1898,702,TampaBay,2,2,2,6,-145.0,-123.0,5.0,...,2021,Montreal,1.0,1.0,1.0,3.0,125.0,113.0,100.0,100.0
17531,1900,705,TampaBay,0,1,1,2,-145.0,-150.0,5.0,...,2021,Montreal,1.0,0.0,1.0,3.0,125.0,135.0,115.0,120.0


In [3]:
# TODO: add divisions, days of rest

In [4]:
nhl_odds_aux = nhl_odds
nhl_odds_aux['home_point_diff'] = nhl_odds_aux.apply(lambda row: row.HomeFinal - row.AwayFinal, axis=1)
nhl_odds_aux['away_point_diff'] = nhl_odds_aux.apply(lambda row: row.AwayFinal - row.HomeFinal, axis=1)


nhl_odds_aux


,Unnamed: 0,Date,AwayTeam,Away1st,Away2nd,Away3rd,AwayFinal,MlAwayOpen,MlAwayClose,OpenOU,...,Home1st,Home2nd,Home3rd,HomeFinal,MlHomeOpen,MlHomeClose,OpenUOdds,CloseUOdds,home_point_diff,away_point_diff
0,0,929,Anaheim,0,0,1,1,-155.0,-123.0,6.0,...,1.0,1.0,2.0,4.0,135.0,103.0,-120.0,-115.0,3.0,-3.0
1,2,930,LosAngeles,0,1,0,1,115.0,132.0,6.0,...,2.0,2.0,0.0,4.0,-135.0,-152.0,-105.0,-132.0,3.0,-3.0
2,4,1003,Anaheim,0,2,0,2,165.0,190.0,5.5,...,1.0,0.0,1.0,3.0,-185.0,-230.0,-125.0,-140.0,1.0,-1.0
3,6,1003,Montreal,1,0,1,3,130.0,140.0,6.0,...,1.0,0.0,1.0,2.0,-150.0,-160.0,-115.0,-130.0,-1.0,1.0
4,8,1003,Ottawa,2,0,1,4,-120.0,-125.0,6.0,...,2.0,1.0,0.0,3.0,100.0,105.0,100.0,115.0,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17528,1894,628,Montreal,0,1,0,1,185.0,180.0,5.0,...,1.0,1.0,3.0,5.0,-225.0,-200.0,100.0,105.0,4.0,-4.0
17529,1896,630,Montreal,0,1,0,1,180.0,177.0,5.0,...,0.0,2.0,1.0,3.0,-220.0,-197.0,100.0,100.0,2.0,-2.0
17530,1898,702,TampaBay,2,2,2,6,-145.0,-123.0,5.0,...,1.0,1.0,1.0,3.0,125.0,113.0,100.0,100.0,-3.0,3.0
17531,1900,705,TampaBay,0,1,1,2,-145.0,-150.0,5.0,...,1.0,0.0,1.0,3.0,125.0,135.0,115.0,120.0,1.0,-1.0


In [56]:
def format_date(date, year):
    datestr = str(date)
    month = datestr[:-2]
    day = datestr[-2:]
    # due to covid, the 2021 season was played entirely in 2021 (no games in oct-dec)
    # and the 2020 season stopped in the middle and resumed in aug/sep
    if(year != 2020 and year != 2021):
        # Game was played from Oct-Dec, ie the previous year
        if(date > 900):
            year -= 1
    elif(year == 2020 and date > 930):
            year -= 1
    return dt.datetime(year, int(month), int(day))

nhl_odds_aux['RealDate'] = nhl_odds_aux.apply(lambda row: format_date(row.Date, row.Year), axis=1)
nhl_odds_aux



6
13


,Unnamed: 0,Date,AwayTeam,Away1st,Away2nd,Away3rd,AwayFinal,MlAwayOpen,MlAwayClose,OpenOU,...,MlHomeClose,OpenUOdds,CloseUOdds,home_point_diff,away_point_diff,home_last_3,away_last_3,home_last_1,away_last_1,RealDate
0,0,929,Anaheim,0,0,1,1,-155.0,-123.0,6.0,...,103.0,-120.0,-115.0,3.0,-3.0,0.0,0.0,0.0,0.0,2007-09-29
1,2,930,LosAngeles,0,1,0,1,115.0,132.0,6.0,...,-152.0,-105.0,-132.0,3.0,-3.0,-3.0,3.0,-3.0,3.0,2007-09-30
2,4,1003,Anaheim,0,2,0,2,165.0,190.0,5.5,...,-230.0,-125.0,-140.0,1.0,-1.0,0.0,0.0,0.0,3.0,2007-10-03
3,6,1003,Montreal,1,0,1,3,130.0,140.0,6.0,...,-160.0,-115.0,-130.0,-1.0,1.0,0.0,0.0,0.0,0.0,2007-10-03
4,8,1003,Ottawa,2,0,1,4,-120.0,-125.0,6.0,...,105.0,100.0,115.0,-1.0,1.0,0.0,0.0,0.0,0.0,2007-10-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17528,1894,628,Montreal,0,1,0,1,185.0,180.0,5.0,...,-200.0,100.0,105.0,4.0,-4.0,8.0,3.0,1.0,1.0,2021-06-28
17529,1896,630,Montreal,0,1,0,1,180.0,177.0,5.0,...,-197.0,100.0,100.0,2.0,-2.0,4.0,0.0,4.0,-4.0,2021-06-30
17530,1898,702,TampaBay,2,2,2,6,-145.0,-123.0,5.0,...,113.0,100.0,100.0,-3.0,3.0,-5.0,7.0,-2.0,2.0,2021-07-02
17531,1900,705,TampaBay,0,1,1,2,-145.0,-150.0,5.0,...,135.0,115.0,120.0,1.0,-1.0,-9.0,9.0,-3.0,3.0,2021-07-05


In [66]:
team_to_games = {}
team_to_last_game_played = {}

for index, row in nhl_odds_aux.iterrows():

    # Update the mapping
    if row.HomeTeam not in team_to_games:
        team_to_games.update({row.HomeTeam : deque([0,0,0])})
        team_to_last_game_played.update({row.HomeTeam : dt.datetime(2007, 7, 1)})

    if row.AwayTeam not in team_to_games:
        team_to_games.update({row.AwayTeam : deque([0,0,0])})
        team_to_last_game_played.update({row.AwayTeam : dt.datetime(2007, 7, 1)})


    last_games = team_to_games.get(row.HomeTeam)
    home_last_3 = last_games[0] + last_games[1] + last_games[2]
    home_last_1 = last_games[0]
    last_games.pop()
    last_games.appendleft(row.home_point_diff)

    last_games = team_to_games.get(row.AwayTeam)
    away_last_3 = last_games[0] + last_games[1] + last_games[2]
    away_last_1 = last_games[0]
    last_games.pop()
    last_games.appendleft(row.away_point_diff)

    nhl_odds_aux.at[index, 'home_last_3'] = home_last_3
    nhl_odds_aux.at[index, 'away_last_3'] = away_last_3
    nhl_odds_aux.at[index, 'home_last_1'] = home_last_1
    nhl_odds_aux.at[index, 'away_last_1'] = away_last_1

    nhl_odds_aux.at[index, 'home_rest'] = (row.RealDate - team_to_last_game_played.get(row.HomeTeam)).days
    nhl_odds_aux.at[index, 'away_rest'] = (row.RealDate - team_to_last_game_played.get(row.AwayTeam)).days

    team_to_last_game_played.update({row.HomeTeam : row.RealDate})
    team_to_last_game_played.update({row.AwayTeam : row.RealDate})


nhl_odds_aux





,Unnamed: 0,Date,AwayTeam,Away1st,Away2nd,Away3rd,AwayFinal,MlAwayOpen,MlAwayClose,OpenOU,...,CloseUOdds,home_point_diff,away_point_diff,home_last_3,away_last_3,home_last_1,away_last_1,RealDate,home_rest,away_rest
0,0,929,Anaheim,0,0,1,1,-155.0,-123.0,6.0,...,-115.0,3.0,-3.0,0.0,0.0,0.0,0.0,2007-09-29,90.0,90.0
1,2,930,LosAngeles,0,1,0,1,115.0,132.0,6.0,...,-132.0,3.0,-3.0,-3.0,3.0,-3.0,3.0,2007-09-30,1.0,1.0
2,4,1003,Anaheim,0,2,0,2,165.0,190.0,5.5,...,-140.0,1.0,-1.0,0.0,0.0,0.0,3.0,2007-10-03,94.0,3.0
3,6,1003,Montreal,1,0,1,3,130.0,140.0,6.0,...,-130.0,-1.0,1.0,0.0,0.0,0.0,0.0,2007-10-03,94.0,94.0
4,8,1003,Ottawa,2,0,1,4,-120.0,-125.0,6.0,...,115.0,-1.0,1.0,0.0,0.0,0.0,0.0,2007-10-03,94.0,94.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17528,1894,628,Montreal,0,1,0,1,185.0,180.0,5.0,...,105.0,4.0,-4.0,8.0,3.0,1.0,1.0,2021-06-28,3.0,4.0
17529,1896,630,Montreal,0,1,0,1,180.0,177.0,5.0,...,100.0,2.0,-2.0,4.0,0.0,4.0,-4.0,2021-06-30,2.0,2.0
17530,1898,702,TampaBay,2,2,2,6,-145.0,-123.0,5.0,...,100.0,-3.0,3.0,-5.0,7.0,-2.0,2.0,2021-07-02,2.0,2.0
17531,1900,705,TampaBay,0,1,1,2,-145.0,-150.0,5.0,...,120.0,1.0,-1.0,-9.0,9.0,-3.0,3.0,2021-07-05,3.0,3.0


In [42]:
def print_stats(money, won, loss, push, sample_set):
    std_dev = statistics.stdev(sample_set)
    predicted_std_dev = std_dev / math.sqrt(len(sample_set))
    mean = statistics.mean(sample_set)
    # expected mean should be 0.5, if we have 50% win/loss
    z_score = (mean-0.5) / predicted_std_dev
    confidence = st.norm.cdf(abs(z_score))
    win_percent = won / (won+loss) * 100

    print ("Ending money is %d" %money)
    print ("Record: %d-%d-%d" %(won, loss, push))
    print ("Win Percent: %f" %win_percent)
    print ("Sample size %d, STD %f, mean %f" %(len(sample_set), std_dev, mean))
    print ("Z-Score %f" %z_score)
    print ("Confidence %f" %confidence)


def print_basic_stats(money, won, loss, push, sample_set):
    std_dev = statistics.stdev(sample_set)
    predicted_std_dev = std_dev / math.sqrt(len(sample_set))
    mean = statistics.mean(sample_set)
    # expected mean should be 0.5, if we have 50% win/loss
    z_score = (mean-0.5) / predicted_std_dev
    confidence = st.norm.cdf(abs(z_score))
    win_percent = won / (won+loss) * 100
    print ("Ending money is %d" %money)
    print ("Record: %d-%d-%d" %(won, loss, push))
    print ("Win Percent: %f" %win_percent)
    print ("Confidence %f" %confidence)

def calculate_profit(odds, wager):
    # -101 and beyond
    if(odds < 0):
        return int((100 / abs(odds)) * wager)
    # +100 and beyond
    return int((odds / 100) * wager)
    

def bet_home(money, odds, won, loss, gross_profit, total_risked):
    total_risked += 300
    if row.home_point_diff > 0:
        money += calculate_profit(odds, 300)
        gross_profit += calculate_profit(odds, 300)
        won += 1
    # No ties in hockey since the full season lockout
    else:
        money -= 300
        loss +=1
    return money, won, loss, gross_profit, total_risked

def bet_away(money, odds, won, loss, gross_profit, total_risked):
    total_risked += 300
    if row.away_point_diff > 0:
        money += calculate_profit(odds, 300)
        gross_profit += calculate_profit(odds, 300)
        won += 1
    # No ties in hockey since the full season lockout
    else:
        money -= 300
        loss += 1
    return money, won, loss, gross_profit, total_risked

def print_stats(money, won, loss, gross_profit, total_risked):
    print('Money left: ' + str(money))
    print('Record: ' + str(won) + '-' + str(loss))
    print('Gross Winnings: ' + str(gross_profit))
    print('Total Risked: ' + str(total_risked))

    net_profit = money - 10000
    print('ROI: ' + str(round(net_profit/total_risked, 2) * 100) + '%')
    print('Norm Winning: ' + str(round(gross_profit/total_risked, 2) * 100) + '%')

def set_vars():
    money = 10000
    won = 0
    loss = 0
    total_risked = 0
    gross_profit = 0
    sample_set = []
    year_to_record = {}
    return money, won, loss, gross_profit, total_risked

In [77]:
money, won, loss, gross_profit, total_risked = set_vars()

for row in nhl_odds_aux.itertuples():

    if (row.MlHomeClose < row.MlAwayClose):
        money, won, loss, gross_profit, total_risked = bet_home(money, row.MlHomeClose, won, loss, gross_profit, total_risked)

print("\nAlways pick the home favourite")
print_stats(money, won, loss, gross_profit, total_risked)

#-------------------------------------

money, won, loss, gross_profit, total_risked = set_vars()

for row in nhl_odds_aux.itertuples():

    if (row.MlAwayClose > row.MlHomeClose):
        money, won, loss, gross_profit, total_risked = bet_away(money, row.MlAwayClose, won, loss, gross_profit, total_risked)

print("\nAlways pick the away underdog")
print_stats(money, won, loss, gross_profit, total_risked)


#-------------------------------------

money, won, loss, gross_profit, total_risked = set_vars()

for row in nhl_odds_aux.itertuples():

    if (row.MlAwayClose > 100):
        money, won, loss, gross_profit, total_risked = bet_away(money, row.MlAwayClose, won, loss, gross_profit, total_risked)

print("\nAlways pick the away underdog with positive odds")
print_stats(money, won, loss, gross_profit, total_risked)


#-------------------------------------

money, won, loss, gross_profit, total_risked = set_vars()

for row in nhl_odds_aux.itertuples():

    if (row.MlAwayClose < row.MlHomeClose ):
        money, won, loss, gross_profit, total_risked = bet_away(money, row.MlAwayClose, won, loss, gross_profit, total_risked)

print("\nAlways pick the away favourite with positive odds")
print_stats(money, won, loss, gross_profit, total_risked)


#-------------------------------------

money, won, loss, gross_profit, total_risked = set_vars()

for row in nhl_odds_aux.itertuples():

    if (row.MlAwayClose > row.MlHomeClose and row.MlAwayClose > row.MlAwayOpen):
        money, won, loss, gross_profit, total_risked = bet_away(money, row.MlAwayClose, won, loss, gross_profit, total_risked)

print("\nAlways pick the away underdog when the line moves against them")
print_stats(money, won, loss, gross_profit, total_risked)


#-------------------------------------

money, won, loss, gross_profit, total_risked = set_vars()

for row in nhl_odds_aux.itertuples():

    if (row.MlAwayClose > row.MlHomeClose and row.MlAwayClose - 100 > row.MlAwayOpen):
        money, won, loss, gross_profit, total_risked = bet_away(money, row.MlAwayClose, won, loss, gross_profit, total_risked)

print("\nAlways pick the away underdog when the line moves against them by at least 100 odds")
print_stats(money, won, loss, gross_profit, total_risked)


#-------------------------------------

money, won, loss, gross_profit, total_risked = set_vars()

for row in nhl_odds_aux.itertuples():

    if (row.MlAwayClose > row.MlHomeClose and row.MlAwayClose - 200 > row.MlAwayOpen):
        money, won, loss, gross_profit, total_risked = bet_away(money, row.MlAwayClose, won, loss, gross_profit, total_risked)

print("\nAlways pick the away underdog when the line moves against them by at least 200 odds")
print_stats(money, won, loss, gross_profit, total_risked)


#-------------------------------------

money, won, loss, gross_profit, total_risked = set_vars()

for row in nhl_odds_aux.itertuples():

    if (row.MlAwayClose > row.MlHomeClose and row.MlAwayClose - 100 > row.MlAwayOpen and row.home_last_3 >= 6):
        money, won, loss, gross_profit, total_risked = bet_away(money, row.MlAwayClose, won, loss, gross_profit, total_risked)

print("\nAlways pick the away underdog when the line moves against them by at least 100 odds and the home team won the last 3 games by >=6")
print_stats(money, won, loss, gross_profit, total_risked)


#-------------------------------------

money, won, loss, gross_profit, total_risked = set_vars()

for row in nhl_odds_aux.itertuples():

    if (row.MlAwayClose > row.MlHomeClose and row.away_rest > row.home_rest):
        money, won, loss, gross_profit, total_risked = bet_away(money, row.MlAwayClose, won, loss, gross_profit, total_risked)

print("\nAlways pick the away underdog when they had more rest than the home team with positive line movement")
print_stats(money, won, loss, gross_profit, total_risked)


#-------------------------------------

money, won, loss, gross_profit, total_risked = set_vars()

for row in nhl_odds_aux.itertuples():

    if (row.away_rest >= 2 and row.home_rest == 1):
        money, won, loss, gross_profit, total_risked = bet_away(money, row.MlAwayClose, won, loss, gross_profit, total_risked)

print("\nAlways pick the away team when they had at least 2 days of rest and home team is B2B")
print_stats(money, won, loss, gross_profit, total_risked)


#-------------------------------------

money, won, loss, gross_profit, total_risked = set_vars()

for row in nhl_odds_aux.itertuples():

    if (row.home_rest >= 2 and row.away_rest == 1):
        money, won, loss, gross_profit, total_risked = bet_home(money, row.MlAwayClose, won, loss, gross_profit, total_risked)

print("\nAlways pick the home team when they had at least 2 days of rest and away team is B2B")
print_stats(money, won, loss, gross_profit, total_risked)


#-------------------------------------

money, won, loss, gross_profit, total_risked = set_vars()

for row in nhl_odds_aux.itertuples():

    if (row.MlHomeClose > row.MlAwayClose and row.home_rest >= 2 and row.away_rest == 1):
        money, won, loss, gross_profit, total_risked = bet_home(money, row.MlAwayClose, won, loss, gross_profit, total_risked)

print("\nAlways pick the home underdog when they had at least 2 days of rest and away team is B2B")
print_stats(money, won, loss, gross_profit, total_risked)


#-------------------------------------

money, won, loss, gross_profit, total_risked = set_vars()

for row in nhl_odds_aux.itertuples():

    if (row.MlHomeClose < row.MlAwayClose and row.home_rest >= 2 and row.away_rest == 1):
        money, won, loss, gross_profit, total_risked = bet_home(money, row.MlAwayClose, won, loss, gross_profit, total_risked)

print("\nAlways pick the home favourite when they had at least 2 days of rest and away team is B2B")
print_stats(money, won, loss, gross_profit, total_risked)


#-------------------------------------


money, won, loss, gross_profit, total_risked = set_vars()

for row in nhl_odds_aux.itertuples():

    if (row.MlHomeClose < row.MlAwayClose and row.home_rest >= 3 and row.away_rest == 1):
        money, won, loss, gross_profit, total_risked = bet_home(money, row.MlAwayClose, won, loss, gross_profit, total_risked)

print("\nAlways pick the home favourite when they had at least 3 days of rest and away team is B2B")
print_stats(money, won, loss, gross_profit, total_risked)


#-------------------------------------


Always pick the home favourite
Money left: -141169
Record: 7334-5114
Gross Winnings: 1383031
Total Risked: 3734400
ROI: -4.0%
Norm Winning: 37.0%

Always pick the away underdog
Money left: -89228
Record: 5114-7334
Gross Winnings: 2100972
Total Risked: 3734400
ROI: -3.0%
Norm Winning: 56.00000000000001%

Always pick the away underdog with positive odds
Money left: -70911
Record: 4432-6611
Gross Winnings: 1902389
Total Risked: 3312900
ROI: -2.0%
Norm Winning: 56.99999999999999%

Always pick the away favourite with positive odds
Money left: -17066
Record: 2656-2015
Gross Winnings: 577434
Total Risked: 1401300
ROI: -2.0%
Norm Winning: 41.0%

Always pick the away underdog when the line moves against them
Money left: -32121
Record: 2718-4036
Gross Winnings: 1168679
Total Risked: 2026200
ROI: -2.0%
Norm Winning: 57.99999999999999%

Always pick the away underdog when the line moves against them by at least 100 odds
Money left: 14068
Record: 378-396
Gross Winnings: 122868
Total Risked: 232200


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f0efbe77-01fa-4860-b5ee-e7eac30d44e8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>